Define a single function that runs the stock buy in recommender

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
def pred_open (stock_list: list, contrib_amt: list = [10], trade_type: str = 'stock', roll_days: int = 65, buyvalue: float = 1.2, multiplier: float = 5):
    
    ### duplicate contrib_amt for all stocks if only 1 listed
    if len(contrib_amt) == len(stock_list):
        pass
    elif len(contrib_amt) == 1: 
        contrib_amt = [contrib_amt[0] for x in enumerate(stock_list)]
    else:
        print('Incorrect length of contrib_amt. Make it match the length of the stock list or be 1 value')
        exit()

    ### pull most recent day
    if trade_type == 'crypto' or trade_type == 'index':
        pass
    else:
        x = 0
        while x < 1:
            df_now = yf.download(
            tickers = stock_list
            ,period = '1d' # set for 'today' instead
            ,interval = '1m'
            )

            # ensures a single stock can pass through, not just 2+ 
            if len(stock_list) == 1:
                df_now[stock_list[0]] = df_now['Open']
                df_now = df_now[[stock_list[0]]]
            else:
                df_now = df_now['Open']

            df_now = df_now.head(1) # open for today
            df_now = df_now.fillna(0)

            x = 1
            for i in stock_list:
                x = x * int(df_now[i])

            if x == 0: # wait 15 seconds if data aren't complete
                time.sleep(15)

    ### Overly complex way to pull data, but I have found that 'Open' prices are just a copy of the previous day for the first few minutes of the trading day
    ### This method pulls in the true Open prices for today much quicker (a couple minutes after 6:30am PST)

    if trade_type == 'crypto' or trade_type == 'index':
        df = yf.download(
            tickers = stock_list
            ,period = str(roll_days) + 'd'
        )

        # ensures a single crypto or index can pass through, not just 2+ 
        if len(stock_list) == 1:
            df[stock_list[0]] = df['Open']
            df = df[[stock_list[0]]]
        else:
            df = df['Open']
    else:
        # Pull all data except for today
        df_bulk = yf.download(
                tickers = stock_list
                ,period = str(roll_days) + 'd'
            )

        # ensures a single stock can pass through, not just 2+ 
        if len(stock_list) == 1:
            df_bulk[stock_list[0]] = df_bulk['Open']
            df_bulk = df_bulk[[stock_list[0]]]
        else:
            df_bulk = df_bulk['Open']

        df_good_index = df_bulk.copy() # used to grab the ideal index
        df_bulk.drop(df_bulk.tail(1).index,inplace=True) # bulk w/o the most recent day

        # join the data (index is still bad)
        df = pd.concat([df_bulk, df_now])

        # sub in a good index
        df = df.reindex_like(df_good_index)

        # sub in good open data for today
        for i in stock_list:
            df[i][len(df)-1] = df_now[i].copy()
        

    # add an index and useable date
    df['Index'] = np.arange(1,len(df)+1)
    df['date'] = df.index

    # error checking, if a stock doesn't have enough history based on the current needs
    nlist = []
    for i in stock_list:
        if pd.isna(df[i].iloc[0]) == True:
            nlist.append(i)

    if len(nlist) >0:
        print('Stocks with not enough history', nlist)
        for j in nlist:
            print(j, 'missing days:', df['Index'].count()-df[j].count())
        exit() # Maybe not the best to add this. I still want to see the data

    # create pred and pred/open list for each of the n dataframes
    pred_open_list = []
    for j in stock_list:
        x = range(1,roll_days+1) # range must be 1-roll_days, not the auto implied 0-(roll_days-1)
        y = df[j]
        m, b = np.polyfit(x, y, 1)
        d = m*roll_days+b

        pred_open_list.append(d / df[j][roll_days-1] * d / df[j][roll_days-1])

    multiplier_list = []
    for i, j in enumerate(stock_list):
        if pred_open_list[i] > buyvalue:
            multiplier_list.append(1)
        else:
            multiplier_list.append(0)

    final_buy_list = []
    for i, j in enumerate(stock_list):
        if multiplier_list[i] == 0:
            final_buy_list.append(contrib_amt[i])
        else:
            final_buy_list.append(round(contrib_amt[i]*pred_open_list[i]*multiplier, 2))

    final_df = pd.DataFrame()
    final_df['stock'] = stock_list
    final_df['buy_in_amt'] = final_buy_list

    trade_day_date = df.tail(1)['date'].item().strftime('%Y.%m.%d')

    # return final_df, print(f'Values calculated for: {trade_day_date}')
    return final_df


In [3]:
pred_open(['AAPL'])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,stock,buy_in_amt
0,AAPL,10


In [4]:
pred_open(['AAPL', 'AMD', 'AMZN', 'CRM', 'GOOG', 'INTC', 'MSFT', 'NVDA', 'QQQ', 'SBUX', 'SQ', 'TSLA', 'TSM'])

[*********************100%***********************]  13 of 13 completed
[*********************100%***********************]  13 of 13 completed


,stock,buy_in_amt
0,AAPL,10
1,AMD,10
2,AMZN,10
3,CRM,10
4,GOOG,10
5,INTC,10
6,MSFT,10
7,NVDA,10
8,QQQ,10
9,SBUX,10


In [5]:
pred_open(['AAPL', 'AMD', 'AMZN', 'CRM', 'GOOG', 'INTC', 'MSFT', 'NVDA', 'QQQ', 'SBUX', 'SQ', 'TSLA', 'TSM'], 
    [5.00, 3.75, 2.50, 1.25, 2.50, 3.75, 2.50, 3.75, 2.50, 2.50, 2.50, 5.00, 3.75], 
    'stock', 65, 1.2, 5)


[*********************100%***********************]  13 of 13 completed
[*********************100%***********************]  13 of 13 completed


,stock,buy_in_amt
0,AAPL,5.00
1,AMD,3.75
2,AMZN,2.50
3,CRM,1.25
4,GOOG,2.50
5,INTC,3.75
6,MSFT,2.50
7,NVDA,3.75
8,QQQ,2.50
9,SBUX,2.50


In [6]:
pred_open(['AAPL', 'AMD', 'AMZN', 'CRM', 'GOOG', 'INTC', 'MSFT', 'NVDA', 'QQQ', 'SBUX', 'SQ', 'TSLA', 'TSM'], [5], 'stock', 65, 1.2, 5)


[*********************100%***********************]  13 of 13 completed
[*********************100%***********************]  13 of 13 completed


,stock,buy_in_amt
0,AAPL,5
1,AMD,5
2,AMZN,5
3,CRM,5
4,GOOG,5
5,INTC,5
6,MSFT,5
7,NVDA,5
8,QQQ,5
9,SBUX,5


In [7]:
pred_open(['AAPL'], [5], 'stock', 65, 1.2, 5)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,stock,buy_in_amt
0,AAPL,5


In [8]:
pred_open(['BTC-USD', 'ETH-USD'], [5, 3.75], 'crypto', 65, 1.2, 5)

[*********************100%***********************]  2 of 2 completed


,stock,buy_in_amt
0,BTC-USD,5.00
1,ETH-USD,3.75


In [9]:
pred_open(['BTC-USD'], [5], 'crypto', 65, 1.2, 5)

[*********************100%***********************]  1 of 1 completed


,stock,buy_in_amt
0,BTC-USD,5
